<a href="https://colab.research.google.com/github/timbosssds/ML/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required libraries
# !pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers
# !pip install pypdf
# !pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 4.6 MB/s eta 0:0

In [3]:
# MileStone 1
# -- end-to-end (No Citations)

# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
list_items = texts
text = convert_list_to_string_with_speech_marks(list_items)

# -- Model
context=text
from transformers import pipeline
def generate_answer(question, context):
  """Generates an answer to a question given a context."""
  question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
  response = question_answerer(question=question, context=text, max_answer_len=1024, top_p=1)
  response = response[list(response)[-1]]
  return response

# -- Call model and produce answer
# Generate an answer to the question
#question = "what is the Eligibility criteria for the Product"
question = "what is the date"

answer = generate_answer(question, context)
print(answer)

1 October 2022


In [4]:
# MileStone 2
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
list_items = texts
text = convert_list_to_string_with_speech_marks(list_items)

# -- Model
context=text
# Define a function to generate an answer to a question with citations
from transformers import pipeline

def generate_answer_with_citations(question, context):
    """Generates an answer to a question given a context and provides citations."""
    question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
    response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
    answer_text = response[list(response)[-1]]
    answer_start = response["start"]
    answer_end = response["end"]

    # Get the passage containing the answer
    start_index = max(0, context.rfind(".", 0, answer_start))
    end_index = min(len(context), context.find(".", answer_end))
    passage_with_answer = context[start_index:end_index].strip()

    # Get the citation (page number) of the passage
    page_number = context.count(".", 0, start_index) + 1

    return {
        "answer": answer_text,
        "passage": passage_with_answer,
        "citation": f"Page {page_number}"
    }

# Example usage
#question = "what is the date"
question = "what commision do you receive"
answer_info = generate_answer_with_citations(question, context)
print("Answer:", answer_info["answer"])
print("Passage:", answer_info["passage"])
print("Citation:", answer_info["citation"])

Answer: via email, by telephone or fax
Passage: . 
However, there are special arrangements in place for some 
products and services where we can receive your instructions 
electronically via email, by telephone or fax
Citation: Page 13


In [ ]:
# -- All below is WIP...

In [21]:
# WIP - trying to get conversation history
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
  # Get the page object
  page = loader.pages[page_num]
  # Extract the texst from the page and add it to the text variable
  text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
  list_items = texts
  text = convert_list_to_string_with_speech_marks(list_items)

# -- 4.0 Maintain conversation history
conversation_history = []

# -- Model
context=text

# Define a function to generate an answer to a question with citations
from transformers import pipeline

def generate_answer_with_citations(question, context):
  """Generates an answer to a question given a context and provides citations."""
  # Check if the question is related to any of the previous questions
  for previous_question, previous_answer in conversation_history:
    if question.startswith(previous_question):
      context = f"{context} {previous_answer}"

  question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
  response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
  answer_text = response[list(response)[-1]]
  answer_start = response["start"]
  answer_end = response["end"]

  # Get the passage containing the answer
  start_index = max(0, context.rfind(".", 0, answer_start))
  end_index = min(len(context), context.find(".", answer_end))
  passage_with_answer = context[start_index:end_index].strip()

  # Get the citation (page number) of the passage
  page_number = context.count(".", 0, start_index) + 1

  # Add the new question and answer to the conversation history
  conversation_history.append((question, answer_text))

  return {
      "answer": answer_text,
      "passage": passage_with_answer,
      "citation": f"Page {page_number}"}

# Example usage
#question = "what is the date" # original question

#question = "what is afca?" # initial q to test follow up question
question = "what number do i contact you on?" # follow up question to see if conversation memory working

#question = "what email do i use for dispute resolution?" # Intial question
#question = "and phone number?" # Intial question # Follow up

answer_info = generate_answer_with_citations(question, context)
print("Answer:", answer_info["answer"])
print("Passage:", answer_info["passage"])
print("Citation:", answer_info["citation"])


Answer: 1300 555 727
Passage: .au/feedback
National Relay Service 
• TTY or Voice: Call 133 677  then as k for 13 2221
• Speak & listen: Call 1300 555 727  then as k for 13 2221
• SMS relay – Text  0423 677 767
For more information, visit the National Relay Service: 
relayservice
Citation: Page 43


In [23]:
conversation_history

[('what number do i contact you on?', '1300 555 727')]

In [22]:
# -- WIP (trying to use langchain memory)
# MileStone 2
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)


# -- New
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()



# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
list_items = texts
text = convert_list_to_string_with_speech_marks(list_items)

# -- Model
context=text
# Define a function to generate an answer to a question with citations
from transformers import pipeline

def generate_answer_with_citations(question, context, memory):
    """Generates an answer to a question given a context and provides citations."""
    question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
    response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
    answer_text = response[list(response)[-1]]
    answer_start = response["start"]
    answer_end = response["end"]

    # -- New
    MessagesPlaceholder(variable_name="chat_history")

    # Get the passage containing the answer
    start_index = max(0, context.rfind(".", 0, answer_start))
    end_index = min(len(context), context.find(".", answer_end))
    passage_with_answer = context[start_index:end_index].strip()

    # Get the citation (page number) of the passage
    page_number = context.count(".", 0, start_index) + 1

    return {
        "answer": answer_text,
        "passage": passage_with_answer,
        "citation": f"Page {page_number}"
    }

# -- New
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Example usage
question = "what number do i contact you on?" # follow up question to see if conversation memory working

#question = "what email do i use for dispute resolution?" # Intial question
#question = "and phone number?" # Intial question # Follow up
answer_info = generate_answer_with_citations(question, context, memory)
print("Answer:", answer_info["answer"])
print("Passage:", answer_info["passage"])
print("Citation:", answer_info["citation"])

Answer: 
FSG
Passage: .  You may request further details of these programmes 
by contacting us within a reasonable time after receiving this 
FSG and before any financial service is provided to you
Citation: Page 25
